# 可选实验室——神经元和层

在这个实验室中，我们将探索神经元/单元和层的内部运作。特别是，实验室将与您在课程1中掌握的模型、回归/线性模型和逻辑模型进行比较。该实验室将介绍Tensorflow并演示如何在该框架中实现这些模型。

<figure>
   <img src="../../images/C2_W1_NeuronsAndLayers.png" style="width:540px;height:200px;" >
</figure>

## 套餐

**TensorFlow和Keras**

Tensorflow是Google开发的机器学习包。2019年，谷歌将Keras集成到Tensorflow中，并发布了Tensorflow 2.0。Keras是François Chollet独立开发的框架，它创建了一个简单的、以层为中心的Tensorflow接口。本课程将使用Keras界面。

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from lab_utils_common import dlc
from lab_neurons_utils import sigmoidnp, plt_linear, plt_logistic

plt.style.use('./deeplearning.mplstyle')
import logging

logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config, )

## 未激活的神经元——回归/线性模型

### 数据集

我们将使用课程1中房价线性回归的示例。

In [2]:
X_train = np.array([[1.0], [2.0]], dtype=np.float32)
Y_train = np.array([[300.0], [500.0]], dtype=np.float32)
fig, ax = plt.subplots(1, 1)
ax.scatter(X_train, Y_train, marker='x', c='r', label="Data Points")
ax.legend(fontsize='xx-large')
ax.set_ylabel('Price (in 1000s of dollars)', fontsize='xx-large')
ax.set_xlabel('Size (1000 sqft)', fontsize='xx-large')
plt.show()

### 回归/线性模型

没有激活的神经元实现的函数与课程1线性回归中的相同：

$$f_{\mathbf{w},b}(x^{(i)})=\mathbf{w}\cdot x^{(i)}+b\tag{1}$$

我们可以用一个神经元或单元定义一层，并将其与熟悉的线性回归函数进行比较。

In [3]:
linear_layer = tf.keras.layers.Dense(units=1, activation='linear', )

让我们检查一下权重。

In [4]:
linear_layer.get_weights()

没有权重，因为权重尚未实例化。让我们在`X_train`中的一个示例中尝试该模型。这将触发权重的实例化。请注意，该层的输入必须是二维的，因此我们将对其进行重塑。

In [5]:
a1 = linear_layer(X_train[0].reshape(1, 1))
print(a1)

结果是一个形状为(1,1)或一个条目的张量（数组的另一个名称）。

现在让我们看看权重和偏差。这些权重随机初始化为小数，偏差默认初始化为零。

In [6]:
w, b = linear_layer.get_weights()
print(f"w = {w}, b={b}")

具有单个输入特征的线性回归模型（1）将具有单个权重和偏差。这与上面`linear_layer`的尺寸相匹配。

权重被初始化为随机值，因此我们将它们设置为一些已知值。

In [7]:
set_w = np.array([[200]])
set_b = np.array([100])
linear_layer.set_weights([set_w, set_b])
print(linear_layer.get_weights())

让我们将方程（1）与层输出进行比较。

In [8]:
a1 = linear_layer(X_train[0].reshape(1, 1))
print(a1)
alin = np.dot(set_w, X_train[0].reshape(1, 1)) + set_b
print(alin)

他们产生相同的价值观！

现在，我们可以使用线性层对训练数据进行预测。

In [9]:
prediction_tf = linear_layer(X_train)
prediction_np = np.dot(X_train, set_w) + set_b

In [10]:
plt_linear(X_train, Y_train, prediction_tf, prediction_np)

## 具有Sigmoid激活的神经元

具有sigmoid激活的神经元/单元实现的函数与课程1逻辑回归中的相同：

$$f_{\mathbf{w},b}(x^{(i)})=g(\mathbf{w}x^{(i)}+b)\tag{2}$$

其中$$g(x)=sigmoid(x)$$

让我们将$w$和$b$设置为一些已知值并检查模型。

### 数据集

我们将使用课程1逻辑回归中的示例。

In [11]:
X_train = np.array([0., 1, 2, 3, 4, 5], dtype=np.float32).reshape(-1, 1)
Y_train = np.array([0, 0, 0, 1, 1, 1], dtype=np.float32).reshape(-1, 1)

In [12]:
pos = Y_train == 1
neg = Y_train == 0
X_train[pos]

In [13]:
pos = Y_train == 1
neg = Y_train == 0
fig, ax = plt.subplots(1, 1, figsize=(4, 3))
ax.scatter(X_train[pos], Y_train[pos], marker='x', s=80, c='red', label="y=1")
ax.scatter(X_train[neg], Y_train[neg], marker='o', s=100, label="y=0", facecolors='none', edgecolors=dlc["dlblue"],
           lw=3)
ax.set_ylim(-0.08, 1.1)
ax.set_ylabel('y', fontsize=12)
ax.set_xlabel('x', fontsize=12)
ax.set_title('one variable plot')
ax.legend(fontsize=12)
plt.show()

### 逻辑神经元

我们可以通过添加sigmoid激活来实现“逻辑神经元”。神经元的功能由上面的（2）描述。

本节将创建一个包含逻辑层的Tensorflow模型，以演示创建模型的替代方法。Tensorflow最常用于创建多层模型。[Sequential](https://keras.io/guides/sequential_model/)模型是构建这些模型的便捷方法。

In [14]:
model = Sequential([tf.keras.layers.Dense(1, input_dim=1, activation='sigmoid', name='L1')])

model.summary()显示模型中的层数和参数数量。该模型中只有一层，并且该层只有一个单元。该单位有两个参数：$w$和$b$。

In [15]:
model.summary()

In [16]:
logistic_layer = model.get_layer('L1')
w, b = logistic_layer.get_weights()
print(w, b)
print(w.shape, b.shape)

让我们将权重和偏差设置为一些已知值。

In [17]:
set_w = np.array([[2]])
set_b = np.array([-4.5])
logistic_layer.set_weights([set_w, set_b])
print(logistic_layer.get_weights())

让我们将方程（2）与层输出进行比较。

In [18]:
a1 = model.predict(X_train[0].reshape(1, 1))
print(a1)
alog = sigmoidnp(np.dot(set_w, X_train[0].reshape(1, 1)) + set_b)
print(alog)

他们产生相同的价值观！

现在，我们可以使用逻辑层和NumPy模型对训练数据进行预测。

In [19]:
plt_logistic(X_train, Y_train, model, set_w, set_b, pos, neg)

上面的阴影反映了sigmoid的输出，其范围从0到1。

# 恭喜！

您构建了一个非常简单的神经网络，并探索了神经元与课程1中的线性回归和逻辑回归的相似之处。